# Pre-requisites

In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=36196c02fcb49b18b7fbbcb71f70f576afad35e82f6bd503a847b8960e840a03
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import gensim
import warnings
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk import everygrams
from nltk.stem import *
import itertools
from scipy import spatial
import collections
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import CrossEncoder
from gensim.models import Word2Vec
import datetime
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

In [ ]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%cd /content/gdrive/MyDrive/Kethan/CVE_CWE_Mapping/Cross_Encoder/

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Kethan/CVE_CWE_Mapping/Cross_Encoder


In [ ]:
df_train = pd.read_csv("./Data/230710_1/LayerClassifier/train.csv")
df_val = pd.read_csv("./Data/230710_1/LayerClassifier/val.csv")

In [ ]:
df_train.shape, df_val.shape

((110363, 6), (27502, 6))

# Model Training

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
labels = {0: 0,
 1: 1}
labels

{0: 0, 1: 1}

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['is_top_layer']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 256, truncation=True,
                                return_tensors="pt") for text in df['cve.descriptions_list_processed']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 5
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 3449/3449 [17:52<00:00,  3.21it/s]


Epochs: 1 | Train Loss:  0.015                 | Train Accuracy:  0.773                 | Val Loss:  0.013                 | Val Accuracy:  0.813


100%|██████████| 3449/3449 [17:49<00:00,  3.22it/s]


Epochs: 2 | Train Loss:  0.010                 | Train Accuracy:  0.870                 | Val Loss:  0.009                 | Val Accuracy:  0.883


100%|██████████| 3449/3449 [17:50<00:00,  3.22it/s]


Epochs: 3 | Train Loss:  0.007                 | Train Accuracy:  0.923                 | Val Loss:  0.008                 | Val Accuracy:  0.911


100%|██████████| 3449/3449 [17:49<00:00,  3.22it/s]


Epochs: 4 | Train Loss:  0.005                 | Train Accuracy:  0.952                 | Val Loss:  0.007                 | Val Accuracy:  0.933


100%|██████████| 3449/3449 [17:49<00:00,  3.23it/s]


Epochs: 5 | Train Loss:  0.004                 | Train Accuracy:  0.968                 | Val Loss:  0.006                 | Val Accuracy:  0.937


In [ ]:
torch.save(model.state_dict(), './ModelObjects/Temp/Data230710_1/model')

In [ ]:
train_samples=[]
for index,row in df_train.iterrows():
    train_samples.append(InputExample(texts=[row['cve.descriptions_list_processed'], row['cwe.descriptions_list']], label=row['is_similar']))
    train_samples.append(InputExample(texts=[row['cwe.descriptions_list'], row['cve.descriptions_list_processed']], label=row['is_similar']))

dev_samples=[]
for index,row in df_val.iterrows():
    dev_samples.append(InputExample(texts=[row['cve.descriptions_list_processed'], row['cwe.descriptions_list']], label=row['is_similar']))
    dev_samples.append(InputExample(texts=[row['cwe.descriptions_list'], row['cve.descriptions_list_processed']], label=row['is_similar']))

In [ ]:
#Define our Cross-Encoder
train_batch_size = 32
num_epochs = 7
model_save_path = './ModelObjects/Temp/training_benchmark-'+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_path

'./ModelObjects/Temp/training_benchmark-2023-06-30_12-01-55'

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

In [ ]:
#evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

In [ ]:
model = CrossEncoder('bert-base-uncased', num_labels=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
model.fit(train_dataloader=train_dataloader,
          #evaluator=evaluator,
          epochs=num_epochs,
          output_path=model_save_path)

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4014 [00:00<?, ?it/s]

In [ ]:
#evaluator(model)
#model.save(model_save_path)

In [ ]:
#df_val = df_corpus[df_corpus['cve.id'].str.contains('CVE-2022')]

In [ ]:
df_val['prediction_similarity_score'] = list(model.predict(df_val[['cve.descriptions_list_processed','cwe.descriptions_list']].values.tolist()))

In [ ]:
df_val['prediction_similarity'] = df_val['prediction_similarity_score'].apply(lambda score: 1 if score>0.5 else 0)

In [ ]:
accuracy = np.mean(df_val['prediction_similarity']==df_val['is_similar'])
accuracy

0.9341275402943238

In [ ]:
df_val.tail(2)

,cve.descriptions_list,cve.weaknesses_list,cwe.descriptions_list,is_similar,cve.descriptions_list_processed,prediction_similarity_score,prediction_similarity
60974,An information disclosure vulnerability was discovered in the directory and file management of Avaya Aura Appliance Virtualization Platform Utilities (AVPU). This vulnerability may potentially allow any local user to access system functionality and configuration information that should only be available to a privileged user. Affects versions 8.0.0.0 through 8.1.3.1 of AVPU.,668,use extern control format string untrust search path uncontrol search path element unquot search path element file directori access extern parti exposur resourc wrong sphere untrust path dll preload binari plant insecur librari load depend confus product expos resourc wrong control sphere provid unintend actor inappropri access resourc,1,inform disclosur vulner wa discov directori file manag avaya aura applianc virtual platform util avpu vulner may potenti allow local user access system function configur inform avail privileg user affect version avpu,0.998644,1
37029,"In Android for MSM, Firefox OS for MSM, QRD Android, with all Android releases from CAF using the Linux kernel, while processing sparse image, uninitialized heap memory can potentially be flashed due to the lack of validation of sparse image block header size.",200,exposur sensit inform unauthor actor observ discrep gener error messag contain sensit inform insert sensit inform log file inform disclosur inform leak side channel attack product expos sensit inform actor explicitli author access inform,1,android msm firefox msm qrd android android releas caf use linux kernel process spars imag uniniti heap memori potenti flash due lack valid spars imag block header size,0.999744,1


In [ ]:
df_val['prediction_similarity'].value_counts()

0    3604
1    3531
Name: prediction_similarity, dtype: int64

In [ ]:
df_val['is_similar'].value_counts()

1    3573
0    3562
Name: is_similar, dtype: int64

In [ ]:
df_val[df_val['prediction_similarity']!=df_val['is_similar']].shape

(470, 7)

In [ ]:
model.save(model_save_path)

In [ ]:
df_val.columns

Index(['cve.descriptions_list', 'cve.weaknesses_list', 'cwe.descriptions_list',
       'is_similar', 'cve.descriptions_list_processed',
       'prediction_similarity_score', 'prediction_similarity'],
      dtype='object')

In [ ]:
df_val.head(1)

,cve.descriptions_list,cve.weaknesses_list,cwe.descriptions_list,is_similar,cve.descriptions_list_processed,prediction_similarity_score,prediction_similarity
64706,"Bonita Web 2021.2 is affected by a authentication/authorization bypass vulnerability due to an overly broad exclude pattern used in the RestAPIAuthorizationFilter. By appending ;i18ntranslation or /../i18ntranslation/ to the end of a URL, users with no privileges can access privileged API endpoints. This can lead to remote code execution by abusing the privileged API actions.",863,incorrect behavior order author pars canonic author bypass user control sql primari key author bypass user control key use non canon url path author decis guessabl captcha incorrect author intern asset expos unsaf debug access level state horizont author authz product perform author check actor attempt access resourc perform action doe correctli perform check allow attack bypass intend access restrict,1,bonita web affect authent author bypass vulner due overli broad exclud pattern use restapiauthorizationfilt append intransl intransl end url user privileg access privileg api endpoint lead remot code execut abus privileg api action,0.986959,1


In [ ]:
df_val[df_val['prediction_similarity']!=df_val['is_similar']][['cve.weaknesses_list','is_similar','prediction_similarity_score']]

,cve.weaknesses_list,is_similar,prediction_similarity_score
891,400,0,0.965081
895,400,0,0.992750
897,400,0,0.992797
3672,200,0,0.557984
3884,200,0,0.967949
...,...,...,...
30541,345,0,0.567938
30988,610,1,0.244335
31131,662,1,0.068080
31132,662,1,0.057853


In [ ]:
df_val.to_excel("")

In [ ]:
#!zip -r ./Cross_Encoder.zip /content/D:/AI_ML/kethan/CVE_CWE_Mapping/Output/CrossEncoder/training_benchmark-2023-06-05_12-22-10/

# Inference

In [ ]:
model = CrossEncoder('./ModelObjects/Temp/training_benchmark-2023-06-20_06-47-15/')

In [ ]:
df_test = df_corpus[(df_corpus['cve.id'].str.contains('CVE-2023')|df_corpus['cve.id'].str.contains('CVE-2022'))
 & (df_corpus['is_similar']==1)]

In [ ]:
df_test.shape

(3445, 6)

In [ ]:
counter=0
def return_predictions(row):
  global counter
  counter+=1
  print('\r Counter value: {0} of {1}'.format(str(counter),str(df_test.shape[0])),end='')
  #print("Counter value: "+str(counter)+" of "+str(df_test.shape[0]))
  similarities ={}
  for key,val in dict_mitre_view1003.items():
      similarities[key] = model.predict([val,row['cve.descriptions_list_processed']])
  sorted_similarities = sorted(similarities.items(), key=lambda x:x[1], reverse=True)
  sorted_similarities = [val[0] for val in sorted_similarities]
  sorted_similarities = sorted_similarities[:10]
  return sorted_similarities

In [ ]:
df_test['predicted_values_lst'] = df_test.apply(lambda row: return_predictions(row),axis=1)

 Counter value: 3445 of 3445

In [ ]:
df_test['predicted_values_rank'] = df_test.apply(lambda row: row['predicted_values_lst'].index(str(row['cve.weaknesses_list'])) if str(row['cve.weaknesses_list']) in row['predicted_values_lst']  else 10,axis=1)

In [ ]:
df_test.head(5)

,cve.id,cve.descriptions_list,cve.weaknesses_list,cwe.descriptions_list,is_similar,cve.descriptions_list_processed,predicted_values_lst,predicted_values_rank
509,CVE-2022-21670,"markdown-it is a Markdown parser. Prior to version 1.3.2, special patterns with length greater than 50 thousand characterss could slow down the parser significantly. Users should upgrade to version 12.3.2 to receive a patch. There are no known workarounds aside from upgrading.",400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,markdown markdown parser prior version special pattern length greater thousand charact could slow parser significantli user upgrad version receiv patch known workaround asid upgrad,"[20, 682, 863, 436, 407, 269, 326, 704, 345, 669]",10
510,CVE-2022-21839,Windows Event Tracing Discretionary Access Control List Denial of Service Vulnerability.,400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,window event trace discretionari access control list denial servic vulner,"[20, 345, 362, 732, 330, 436, 287, 668, 706, 610]",10
511,CVE-2022-21843,"Windows IKE Extension Denial of Service Vulnerability. This CVE ID is unique from CVE-2022-21848, CVE-2022-21883, CVE-2022-21889, CVE-2022-21890.",400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,window ike extens denial servic vulner cve id uniqu cve cve cve cve,"[20, 345, 119, 436, 672, 754, 269, 669, 704, 682]",10
512,CVE-2022-21847,Windows Hyper-V Denial of Service Vulnerability.,400,uncontrol resourc consumpt alloc resourc without limit throttl miss refer activ alloc resourc miss refer activ file descriptor handl alloc file descriptor handl without limit throttl log excess data memori alloc excess size valu improp restrict power consumpt incorrect use autobox unbox perform critic oper improp write handl limit write non volatil memori improperli control sequenti memori alloc resourc exhaust file descriptor exhaust stack exhaust stack exhaust product doe properli control alloc mainten limit resourc therebi enabl actor influenc amount resourc consum eventu lead exhaust avail resourc,1,window hyper v denial servic vulner,"[20, 345, 400, 119, 436, 672, 754, 404, 682, 665]",2
513,CVE-2022-21848,"Windows IKE Extension Denial of Service Vulnerability. This CVE ID is unique from CVE-2022-21843, CVE-2022-21883, CVE-2022-21889, CVE-2022-218

In [ ]:
limit=10
scores = pd.DataFrame()
scores['Rank']= range(0,limit)

def return_accuracy(counter,limit,levelName='Unnamed'):
    global scores
    for top_rank_limit in range(limit):
        correct = np.sum([i[1] for i in counter.items()  if i[0]<=top_rank_limit])
        total = np.sum([i[1] for i in counter.items()])
        accur = correct/total
        scores.loc[top_rank_limit,levelName] = accur

return_accuracy(collections.Counter(df_test['predicted_values_rank']),limit)
scores

,Rank,Unnamed
0,0,0.341074
1,1,0.468795
2,2,0.545718
3,3,0.592743
4,4,0.629318
5,5,0.664731
6,6,0.689695
7,7,0.719884
8,8,0.741945
9,9,0.764296


In [ ]:
df_test.to_csv('./Output/Temp/220620_22_23_Predictions.csv',index=False)